# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [2]:
import torch
import torchvision
from torch import nn



# !pip install gdown==3.6.0
# my_file_id = '1_tklnrqxPMkv21A8PkWPk-x0i4dRliNV'
# !gdown https://drive.google.com/uc?id={my_file_id}
# !ls
# !wget https://transfer.sh/6GjUs/goog.pt
# !wget https://transfer.sh/Yl4A3/goog_prev.pt
# !wget https://transfer.sh/Gw6ls/goog1.pt

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model1 = torchvision.models.resnet152(pretrained=False)
        in_features = self.model1.fc.in_features
        out_features = 102
        fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512, out_features)
        )
        self.model1.fc = fc
        self.model2 = torchvision.models.resnet152(pretrained=False)
        fc = nn.Linear(in_features, out_features)
        self.model2.fc = fc
        self.relu = nn.ReLU(0.2)
#         self.linear = nn.Linear(102*2,64)
#         self.linear1 = nn.Linear(64,102)
        self.model1.load_state_dict(torch.load('goog1.pt',map_location='cpu'), strict=False)
        self.model2.load_state_dict(torch.load('goog.pt',map_location='cpu'), strict=False)
        self.model1.requires_grad = False
        self.model2.requires_grad = False

    def forward(self, x):
        m1 = self.model1(x)
        m2 = self.model2(x)
#         m1,_ = torch.max(self.model1(x),1)
#         m2,_ = torch.max(self.model2(x),1)
#         x = torch.cat((m1, m2), 1)
#         return self.linear1(nn.Dropout(0.2)(self.linear(x)))
        return (m1+m2)/2



model = Model()



# in_features = model.fc.in_features
# # in_features = model.classifier[0].in_features
# out_features = 102

# fc = nn.Sequential(
#     nn.Linear(in_features, 512),
#     nn.Dropout(0.2),
#     nn.ReLU(),
#     nn.Linear(512, out_features)
# )

# model.fc = fc

# model.load_state_dict(torch.load('goog1.pt', map_location='cpu'), strict=False)

model.cpu()

# Load your model to this variable
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]